# Erstellen einer lokalen Github Datenbank

Um lokale Anfragen machen zu können, muss zuerst eine lokale Datenbank erzeugt werden. Im folgenden wird gezeigt, wie diese Datenbank erzeugt wird.

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

from data_generation.generate_local_github_database import download_and_save_git_reposiories_according_to_keyword_without_embedding

# Topics included
Wir laden die Github-Tools aus einer Sammlung von Github-Repositories herunter, die auf den folgenden Schlüsselwörtern basieren. Für jedes Schlüsselwort werden 20 Tools heruntergeladen. Wir extrahieren die Informationen aller heruntergeladenen Tools und speichern sie in der Datei "repositories_without_embd" im Datenordner.
Die extrahierten Informationen umfassen:
- Link zum Werkzeug
- Link zur Werkzeugbeschreibung
- Vorverarbeitete Werkzeugbeschreibungen

Laut Statistik gibt es im Mai 2021 170 Millionen verschiedene Repositories auf Github. Der Prozentsatz derer, die wir gespeichert haben, ist also im Verhältnis zur Gesamtzahl gering.

In [ ]:
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "data mining", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Image and Speech Recognition", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Natural Language Processing", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Fraud Detection", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Recommendation Systems", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Healthcare", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Plotting", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Dataset", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Tools", num = 20, out_path = './data/repositories_without_embd.csv')
df = download_and_save_git_reposiories_according_to_keyword_without_embedding(keyword = "Reinforcement Learning", num = 20, out_path = './data/repositories_without_embd.csv')

In [ ]:
df = pd.read_csv('./data/repositories_without_embd.csv', index_col = 0)

In [ ]:
df

In [ ]:
pd.concat([df]*10).reset_index().iloc[:, 1:].to_csv("data/copydata.csv")

# Erzeugung der Wordembeddings

Wordembeddings sind eine Technik des maschinellen Lernens, die verwendet wird, um Wörter oder Sätze in einem Vektorraum darzustellen. Diese Darstellung ermöglicht es, semantische Beziehungen zwischen Wörtern zu erfassen, indem ähnliche Wörter nahe beieinander liegen. Ein häufig verwendetes Modell zur Erzeugung von Wordembeddings ist Word2Vec.

Die grundlegende Idee hinter Wordembeddings besteht darin, dass Wörter, die in ähnlichen Kontexten auftreten, ähnliche Vektordarstellungen erhalten. Zum Beispiel könnten die Wörter "Katze" und "Hund" ähnliche Vektoren haben, da sie oft in ähnlichen Kontexten, wie "Haustier", "Tierarzt" usw., auftreten. Diese Vektordarstellungen sind in der Regel hochdimensional, wobei jede Dimension einem bestimmten Merkmal oder Kontext entspricht.

Durch die Berechnung der Cosinus Similarity zwischen den Vektordarstellungen von zwei Wörtern oder Sätzen kann ihre semantische Ähnlichkeit bestimmt werden. Je näher die Kosinusähnlichkeitswert an 1 liegt, desto ähnlicher sind die Wörter oder Sätze.

Im folgenden werden basierend auf der Beschreibung der Github Repositories Wordembeddings erzeugt und in der lokalen Datenbank hinzugefügt.

In [ ]:
import numpy as np
import time

from localgpt_request import get_text_embedding_with_localgpt, get_docu_embedding_with_localgpt, get_docu_embedding_save_to_chroma_with_localgpt
from gpt_request import get_text_embedding_with_openai

## Anfragen mit dem Local GPT
Das hier verwendete Modell ist instructor-xl. Die Modellgröße beträgt etwa 5G.

In [ ]:
start_time = time.time()
get_docu_embedding_with_localgpt(df.iloc[:10, 2].values, model_name = "hkunlp/instructor-xl")
end_time = time.time()
use_time = end_time - start_time

print(f"Die Gesamtzeit für die Erstellung der ersten 10 Embeddings beträgt：{use_time}")
print(f"Die durchschnittliche Zeit für die Erstellung der ersten 10 Embeddings beträgt：{use_time/10}")

## Anfragen mit dem OpenAI GPT

In [ ]:
start_time = time.time()
list_time = []
for i in range(10):
    get_text_embedding_with_openai(df.iloc[i, 2])
    end_time = time.time()
    use_time = end_time - start_time
    list_time.append(use_time)

print(f"Die Gesamtzeit für die Erstellung der ersten 10 Embeddings beträgt：{np.sum(list_time)}")
print(f"Die durchschnittliche Zeit für die Erstellung der ersten 10 Embeddings betrug：{np.mean(list_time)}")

## Beschleunigung mit Slurm
Ein Problem beim Multithreading ist, dass das Laden des Modells in jedem Thread sehr viel Zeit in Anspruch nehmen kann. Dies kann zwar durch den Fork-Mechanismus gehandhabt werden, aber Fork unterstützt keine Cuda-Kontextkomplexität und kann zu Fehlern führen.

In [ ]:
import subprocess
import sys
sys.path.append("/pfs/data5/home/kit/tm/hj7422/2023_sdsc_catalog/sdsc-cataglog/sdsc-cataglog")

In [ ]:
def submit_slurm_job(script_file):
    # Slurm-Befehl erstellen
    cmd = './' + script_file  
    
    # Ausführen des Slurm-Befehls mit Hilfe des Subprozess-Moduls
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)

    # Abfrage der Ausgabe einer Aufgabe, bis die Aufgabe abgeschlossen ist
    while True:
        output = proc.stdout.readline()
        if output == b'' and proc.poll() is not None:
            break
        if output:
            print(output.decode().strip())
    
    # Abrufen des Rückgabestatuscodes der Aufgabe
    return_code = proc.poll()
    print('Aufgabenrückgabe-Statuscode:', return_code)


In [ ]:
submit_slurm_job('run_db/run_db_embd.sh')

# TODO Was passiert hier?

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(index = ["DSADS", "HAPT", "OPPO", "PAMAP2", "RW"], columns = ["MCNN_base", "DCL_base", "Transformer_base", "MCNN_activityGAN", "DCL_activityGAN", "Transformer_activityGAN", "MCNN_optiHAR", "DCL_optiHAR", "Transformer_optiHAR"])

In [ ]:
df["MCNN_base"] = [82.0, 78.6, 39.4, 69.8, 68.8]
df["DCL_base"] = [85.2, 80.3, 39.8, 73.4, 70.5]
df["Transformer_base"] = [80.4, 79.5, 38.7, 73.5, 68.2]
df["MCNN_activityGAN"] = [85.4, 81.0, 40.6, 72.6, 76.0]
df["DCL_activityGAN"] = [86.4, 81.5, 38.7, 76.6, 74.3]
df["Transformer_activityGAN"] = [83.5, 81.5, 38.7, 76.5, 74.2]
df["MCNN_optiHAR"] = [90.6, 83.2, 43.4, 79.2, 80.0]
df["DCL_optiHAR"] = [91.2, 84.8, 44.4, 78.9, 78.4]
df["Transformer_optiHAR"] = [87.5, 83.1, 45.5, 75.5, 75.9]

In [ ]:
df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# 对数据按行进行标准化
df_normalized = df.div(df.max(axis=1), axis=0)

plt.figure(figsize=(10, 8))
sns.heatmap(df_normalized, annot=df, fmt='', cmap="YlGnBu", cbar=False)
#plt.show()
plt.savefig("heatmap.pdf", bbox_inches='tight', dpi=300)
plt.close()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df, annot=True, cmap="YlGnBu")
plt.show()


In [ ]:
df.plot(kind='bar', figsize=(12, 8))
plt.ylabel('Performance')
plt.title('Performance of Different Models and Methods on Different Datasets')
plt.show()


In [ ]:
import plotly.graph_objects as go

# 先分离出模型和提升方法名称，使其更具可读性
df.columns = pd.MultiIndex.from_tuples([tuple(c.split('_')) for c in df.columns])

fig = go.Figure()

# 为每个模型-提升方法组合创建一个新的条形图系列
for model_method in df.columns:
    fig.add_trace(go.Bar(
        name='_'.join(model_method),
        x=df.index,
        y=df[model_method],
        offsetgroup='_'.join(model_method),
        hoverinfo='name+y'
    ))

fig.update_layout(
    title='Performance of Different Models and Methods on Different Datasets',
    yaxis_title='Performance',
    xaxis_title='Dataset',
    barmode='group',  # 将每个数据集的模型-方法组合分组
    bargap=0.15,      # 每个分组的间隔大小
    bargroupgap=0.1   # 分组内条形图的间隔大小
)

fig.show()


In [ ]:
df

In [ ]:
a = (df['MCNN_optiHAR']- df['MCNN_activityGAN'])/df['MCNN_activityGAN']
b = (df['DCL_optiHAR']- df['DCL_activityGAN'])/df['DCL_activityGAN']
c = (df['Transformer_optiHAR']- df['Transformer_activityGAN'])/df['Transformer_activityGAN']

In [ ]:
(a+b+c)/3

In [ ]:
sum([0.054783, 0.029094, 0.035954, 0.043575])/4

In [ ]:
import openml

In [ ]:
datasets_list = openml.datasets.list_datasets(output_format='dataframe')

In [ ]:
datasets_list

In [ ]:
import requests

def get_data_description(data_id):
    url = f"https://www.openml.org/api/v1/data/{data_id}"
    response = requests.get(url)
    data = response.json()
    return data

# Specify the data_id of the dataset for which you want to retrieve the description
data_id = 45578  # Example data_id, you should replace it with the actual ID of the dataset you want to retrieve

data_description = get_data_description(data_id)
print(data_description)

In [ ]:
response = requests.get('https://www.openml.org/api/v1/data/2')
data = response.json()
data

In [ ]:
response

In [ ]:
dataset = openml.datasets.get_dataset(1)

# 获取数据集的详细信息
print(dataset.description)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/openml_dataset_info.csv')
df

In [ ]:
df['description'][0]